# Convergence speed analysis

This notebook assumes the user is in his own machine.

**Change this if necessary:**

In [2]:
USER = "st76i1_5"
GROUP = "st76i1"
PASSWORD = input('Password: ')

In [3]:
import sys
sys.path.append('/home/viniciustmoraes/Desktop/ST7Project/propagation-code-optimization') # change this

### Generate runs

In [4]:
# make upload
!sshpass -p {PASSWORD} \
    scp -r ../../optimizer/ {USER}@chome.metz.supelec.fr:/usr/users/{GROUP}/{USER}/


In [5]:
# Defines the number of times an algorithm is repeated for statistical analysis
n_stat = int(input('Number of simulations for statistical analysis: '))          

# run algorithm on cluster
runs = [
    "--algorithm hill_climbing --steps 100",
    #"--algorithm greedy --steps 10",
    #"--algorithm tabu_greedy --steps 10 --hparams '{\"n_tabu\":5}'",
    #"--algorithm simulated_annealing --steps 100 --hparams '{\"t0\":20}'",
    #"--algorithm csa --steps 100",
    #"--algorithm cmaes --steps 100",
]

repeated_runs = []

for i in runs:
    repeated_runs += [i]*n_stat



In [6]:

import pexpect

child = pexpect.spawn(f"sshpass -p {PASSWORD} \
                      ssh -tt {USER}@chome.metz.supelec.fr \
                      srun -p cpu_tp -N 1 -n 32 --pty bash",
                      timeout=5*60)

prompt = f"{USER}@kyle[0-9]+:"

for i in range(len(repeated_runs)):
    occupied = child.expect([prompt, "srun: job [1-9]+ queued and waiting for resources"])

    if occupied == 0:
        #cmd = f"python3 -m optimizer.main {repeated_runs[i]} --log convergence_alg{i//n_stat}_run{i%n_stat}.log"
        cmd = f"python3 -m optimizer.main {repeated_runs[i]} --log convergence_alg{0}_run{1}.log"
        child.sendline(cmd)
        running = 1
        while running == 1:
            running = child.expect(['Run finished', '\n'])
            output = child.before.decode()
            print(output)
        output = child.after.decode()
        print(output)

    elif occupied == 1:
        raise Exception("Error: no resource available. Try running `mysrun` and `scancel` on the remote machine.")
    
child.close()


 ~st76i1_5@kyle63:~$ python3 -m optimizer.main --algorithm hill_climbing --steps 100 --log convergence_alg0_run1.log
MPI startup(): PMI server not found. Please set I_MPI_PMI_LIBRARY variable if it is not a singleton case.
[info] [Me=0] Args:
	algorithm: hill_climbing
	batch: False
	flexible_shape: False
	hparams: {}
	log: convergence_alg0_run1.log
	phase: deploy
	problem_size: [256, 256, 256]
	program_path: iso3dfd-st7
	seed: 33
	steps: 100
	use_energy: False
[info] [Me=0] Hyperparameters:
[info] [Me=0] real seed: 33
[info] [Me=0] Executed command: /usr/bin/mpirun -np 1 -map-by ppr:1:node:PE=16 python3 -m optimizer.main --algorithm hill_climbing --hparams '{}' --log convergence_alg0_run1.log --phase run --problem_size 256 256 256 --program_path iso3dfd-st7 --seed 33 --steps 100 
Unable to init server: Could not connect: Connection refused
Unable to init server: Impossible de se connecter : Connection refused

(main.py:179545): Gdk-CRITICAL **: 22:48:45.793: gdk_cursor_new_for_display

In [ ]:
# make get_logs
!mkdir -p tmp/
!sshpass -p {PASSWORD} \
    scp '{USER}@chome.metz.supelec.fr:/usr/users/{GROUP}/{USER}/convergence_alg*.log' ./tmp/

### Convert logs to data

In [ ]:
from optimizer.logger import log_to_list
import numpy as np

data = {}
alg_to_name = {0:'hillclimbing', 1:'greedy', 2:'tabu', 3:'simulated_annealing', 4:'csa'}

for i in range(n_stat*len(runs)):
    alg = i//2
    run = i%2
    alg_name = alg_to_name[alg]
    
    data[alg_name+str(run)] = log_to_list(f'tmp/convergence_alg{alg}_run{run}.log')


### Plot graphs

In [ ]:
# Extract data with flairs (either Initial or New best) -> Hill Climbing and Simulated Annealing
new_data = {}

for i in range(n_stat):
    new_data[f'hillclimbing{n_stat}'] = []
    new_data[f'simulated_annealing{n_stat}'] = []

for key in data:
    if 'hillclimbing' in key or 'simulated_annealing' in key:
        for line in data[key]:
            if 'flair' in line:
                new_data[key].append(line)
    else:
        new_data[key] = data[key]
                
# Starts x and y variables for all the algorithms 

x0,y0 = [],[]
x1,y1 = [],[]
x2,y2 = [],[]
x3,y3 = [],[]
x4,y4 = [],[]
x5,y5 = [],[]

# For each iteration of each algorithm, writes the x (Nmbr of evaluations) and y (cost function) values

for k in range(n_stat):
    y0 += [float(new_data[f'hillclimbing{k}'][i]['Cost']) for i in range(0, len(new_data[f'hillclimbing{k}']))]
    x0 += [float(new_data[f'hillclimbing{k}'][i]['eval']) for i in range(0, len(new_data[f'hillclimbing{k}']))]

    y1 += [float(new_data[f'greedy{k}'][i]['Cost']) for i in range(0, len(new_data[f'greedy{k}']))]
    x1 += [float(new_data[f'greedy{k}'][i]['eval']) for i in range(0, len(new_data[f'greedy{k}']))]

    y2 += [float(new_data[f'tabu{k}'][i]['Cost']) for i in range(0, len(new_data[f'tabu{k}']))]
    x2 += [float(new_data[f'tabu{k}'][i]['eval']) for i in range(0, len(new_data[f'tabu{k}']))]

    y3 += [float(new_data[f'simulated_annealing{k}'][i]['Cost']) for i in range(0, len(new_data[f'simulated_annealing{k}']))]
    x3 += [float(new_data[f'simulated_annealing{k}'][i]['eval']) for i in range(0, len(new_data[f'simulated_annealing{k}']))]

    y4 += [float(new_data[f'csa{k}'][i]['Cost']) for i in range(0, len(new_data[f'csa{k}']))]
    x4 += [float(new_data[f'csa{k}'][i]['eval']) for i in range(0, len(new_data[f'csa{k}']))] 

### Statistical Treatement

In [ ]:

for (x,y) in [(x0,y0),(x1,y1),(x2,y2),(x3,y3),(x4,y4)]:
    last_eval = min([max(x[i]) for i in range(len(x))])     # Determines which is the first limiting value for x, among the different runs
    
    # Define the common set of x values
    x_common = np.arange(0, last_eval, 10)

    # Initialize an array to store the interpolated y values
    y_interp = np.zeros((len(x_common), n_stat))












### Convergence speed comparison per number of evaluations

In [ ]:
from matplotlib import pyplot as plt

plt.plot(x0, y0, label='Hill Climbing')
plt.plot(x1, y1, label='Greedy Hill Climbing')
plt.plot(x2, y2, label='Tabu-Greedy Hill Climbing')
plt.plot(x3, y3, label='Simulated Annealing')
plt.legend()
plt.xlabel("No. of evaluations")
plt.ylabel("Cost (throughput)")
plt.title("Algorithm convergence speed comparison")
plt.grid()